In [23]:
import pandas as pd
import numpy as np
from pandas_datareader import data as pdr
from datetime import date, timedelta
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from stock_history import calc_dates
from stock_history import price_history
from stock_history import plot_stocks 
from statsmodels.tsa.stattools import adfuller
from pmdarima import auto_arima
from statsmodels.tsa.arima.model import ARIMA
import warnings
import pickle

In [2]:
#query y-finance API for last 365 days of trading price for different stocks and bonds 
MSFT = price_history(["MSFT"], 365)[0]
AAPL = price_history(["AAPL"], 365)[0]
TSLA = price_history(["TSLA"], 365)[0]
Berkshire_Hathaway = price_history(["BRK-A"], 365)[0]
BAC = price_history(["BAC"], 365)[0]
GOOG = price_history(["GOOG"], 365)[0]
S_and_P = price_history(["^GSPC"], 365)[0]
PANW = price_history(["PANW"], 365)[0]
BBY = price_history(["BBY"], 365)[0]
WMT = price_history(["WMT"], 365)[0]

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta inste

In [15]:
#concatenated all datasets into one dataframe and ensured all datasets are datetime format 
#so as to train the model on multiple different datasets
Price_histories = [MSFT, AAPL, TSLA, Berkshire_Hathaway, BAC, GOOG, S_and_P, PANW, BBY, WMT]
Combined_Price_histories = pd.concat(Price_histories)

In [17]:
#use pmdarima module to calculate Autoregressive Integrated and Moving Average values for data
stepwise_fit = auto_arima(Combined_Price_histories["Close"], suppress_warnings=True)           
stepwise_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 2510
Model:               SARIMAX(0, 1, 0)   Log Likelihood              -27804.779
Date:                Fri, 29 Mar 2024   AIC                          55611.557
Time:                        18:35:45   BIC                          55617.385
Sample:                             0   HQIC                         55613.673
                               - 2510                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
sigma2      2.472e+08   2.68e+05    920.736      0.000    2.47e+08    2.48e+08
===================================================================================
Ljung-Box (L1) (Q):                   0.01   Jarque-Bera (JB):         190202503.85
Prob(Q):                              0.94   Prob(JB):                         0.00
Heteroskedasticity (H):               0.00   Skew:                           -16.09
Prob(H) (two-sided):                  0.00   Kurtosis:                      1351.47
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [22]:
#trained an ARIMA model 
model = ARIMA(Combined_Price_histories["Close"],order=(0,1,0))
Trained_Model = model.fit()
Trained_Model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                  Close   No. Observations:                 2510
Model:                 ARIMA(0, 1, 0)   Log Likelihood              -27804.779
Date:                Fri, 29 Mar 2024   AIC                          55611.557
Time:                        19:20:19   BIC                          55617.385
Sample:                             0   HQIC                         55613.673
                               - 2510                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
sigma2      2.472e+08   2.68e+05    920.736      0.000    2.47e+08    2.48e+08
===================================================================================
Ljung-Box (L1) (Q):                   0.01   Jarque-Bera (JB):         190202503.85
Prob(Q):                              0.94   Prob(JB):                         0.00
Heteroskedasticity (H):               0.00   Skew:                           -16.09
Prob(H) (two-sided):                  0.00   Kurtosis:                      1351.47
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [24]:
#saved trained model using pickle 
with open('Combined_Price_histories.pkl', 'wb') as f:
    pickle.dump(Combined_Price_histories, f)

In [ ]:
#use the following code to download the model in a new python file 
with open('Combined_Price_histories.pkl', 'rb') as f:
    loaded_model = pickle.load(f)
